In [1]:
print('hello')
      

hello


In [12]:

!wget https://github.com/chrisdgenius/llm-zoomcamp-rag-workshop/blob/main/documents.json


--2024-06-15 23:22:14--  https://github.com/chrisdgenius/llm-zoomcamp-rag-workshop/blob/main/documents.json
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘documents.json.1’

documents.json.1        [ <=>                ]   1.27M  --.-KB/s    in 0.1s    

2024-06-15 23:22:15 (10.5 MB/s) - ‘documents.json.1’ saved [1329993]



In [13]:
!head documents.json


[
  {
    "course": "data-engineering-zoomcamp",
    "documents": [
      {
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?"
      },
      {


In [15]:
import json

with open('./documents.json', 'rt') as f_in:
    documents_file = json.load(f_in)

documents = []

for course in documents_file:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [17]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [14]:
len(documents)


NameError: name 'documents' is not defined

In [7]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '5ea8be723799', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'IRPmiomORja9r_9hgWBugA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [12]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
response = es.indices.create(index=index_name, body=index_settings)

response

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [13]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

NameError: name 'documents' is not defined

To querry elastic search


The given query is an Elasticsearch query designed to search a database for documents relevant to the user's question about joining a course after it has started. Let's break down each component of this query to understand its purpose and function:

### Components of the Elasticsearch Query

1. **Top-level Structure**
    ```json
    {
        "size": 5,
        "query": { ... }
    }
    ```
    - `size: 5`: This indicates that the query should return up to 5 results.

2. **Query Section**
    ```json
    "query": {
        "bool": { ... }
    }
    ```
    - The `bool` (Boolean) query allows combining multiple query clauses with boolean logic.

3. **Must Clause**
    ```json
    "must": {
        "multi_match": { ... }
    }
    ```
    - The `must` clause specifies conditions that must be met for a document to be considered a match.

4. **Multi-Match Query**
    ```json
    "multi_match": {
        "query": user_question,
        "fields": ["question^3", "text", "section"],
        "type": "best_fields"
    }
    ```
    - `query: user_question`: This is the user’s query, "How do I join the course after it has started?"
    - `fields: ["question^3", "text", "section"]`: Specifies the fields to search in the documents. The `question` field is boosted by 3 (`^3`), meaning matches in this field are considered more important.
    - `type: "best_fields"`: This type indicates that the query will find the single best matching field among those specified.

5. **Filter Clause**
    ```json
    "filter": {
        "term": {
            "course": "data-engineering-zoomcamp"
        }
    }
    ```
    - The `filter` clause restricts the results to documents that contain the term "data-engineering-zoomcamp" in the `course` field. This does not affect the score of the results but ensures that only relevant documents are considered.

### Overall Function

The query is designed to search for up to 5 documents that:
- Contain information relevant to the question "How do I join the course after it has started?"
- Give higher importance to matches in the `question` field.
- Only include documents related to the "data-engineering-zoomcamp" course.

### Summary

The query leverages a combination of relevance scoring (via `multi_match` on different fields) and filtering (to restrict results to a specific course) to find the most pertinent answers to the user's question about joining a course that has already started.

In [10]:
user_question = "How do I join the course after it has started?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [9]:
response = es.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")

NameError: name 'index_name' is not defined